In [1]:
import os
  # accessible as a variable in index.html:
from sqlalchemy import *
from sqlalchemy.pool import NullPool
from flask import Flask, request, render_template, g, redirect, Response,url_for

try:
    
    tmpl_dir = os.path.join(os.path.dirname(os.path.abspath(__file__)), 'templates')

except NameError:
    # Fallback for interactive environments like Jupyter notebooks
    # Manually specify the path to your 'templates' directory here
    tmpl_dir = os.path.join(os.getcwd(), 'templates')
    
#app = Flask(__name__, template_folder=tmpl_dir)


In [2]:
DATABASE_USERNAME = "hz2906"
DATABASE_PASSWRD = "763092Kgb!"
DATABASE_HOST = "35.212.75.104" # change to 34.28.53.86 if you used database 2 for part 2
DATABASEURI = f"postgresql://{DATABASE_USERNAME}:{DATABASE_PASSWRD}@{DATABASE_HOST}/proj1part2"

In [3]:
engine = create_engine(DATABASEURI)

In [4]:
with engine.connect() as conn:
    delete_table_command="""
    DROP TABLE test;
    """
    res = conn.execute(text(delete_table_command))
    
    create_table_command = """
    CREATE TABLE IF NOT EXISTS test (
    id serial,
    name text
    )
    """
    res = conn.execute(text(create_table_command))
    insert_table_command = """INSERT INTO test(name) VALUES ('grace hopper'), ('alan turing'), ('ada lovelace')"""
    res = conn.execute(text(insert_table_command))
    conn.commit()
    # you need to commit for create, insert, update queries to reflect

In [5]:
with engine.connect() as conn:
    cursor = conn.execute(text("select * FROM test"))
    record = cursor.fetchone()
    second_record = cursor.fetchone()
    #print(record)
    for row in cursor:
        print (list(row))



[3, 'ada lovelace']


In [6]:
from flask import Flask
from flask import session


app = Flask(__name__)

app.secret_key = 'bikefan'

@app.before_request
def before_request():
    """
    This function is run at the beginning of every web request 
    (every time you enter an address in the web browser).
    We use it to setup a database connection that can be used throughout the request.

    The variable g is globally accessible.
    """
    try:
        g.conn = engine.connect()
    except:
        print("uh oh, problem connecting to database")
        import traceback; traceback.print_exc()
        g.conn = None
@app.teardown_request
def teardown_request(exception):
    """
    At the end of the web request, this makes sure to close the database connection.
    If you don't, the database could run out of memory!
    """
    try:
        g.conn.close()
    except Exception as e:
        pass

@app.route('/', methods=["POST", "GET"])
def home():
    recipes = []
    if request.method == "POST":
        if request.form["type"] == 'email':
            email = request.form["email"]

            emails = []
            select_query = "SELECT user_id from Users where email = :email"
            cursor = g.conn.execute(text(select_query), {'email': email})
            
            for result in cursor:
                emails.append(result[0])
                
                # print(result)
            cursor.close()
            context = dict(data = emails)
            # print(context)
            if len(context['data']) == 0:
                print('email not exist')
            else:
                session['id'] = context['data'][0]
                session['user_email'] = email
                print(session['id'])
        if request.form["type"] == 'search':
            recipe_name = request.form["recipe_name"]
            select_query = "select  * from Recipes where lower(recipe_name) like :recipe_name"
            cursor = g.conn.execute(text(select_query), {'recipe_name': '%' + recipe_name.lower() + '%'})

            for result in cursor:
                dict_ = {'recipe_id':result[0], 'recipe_name':result[1], 'cooking_time':result[2], 'cooking_difficulty':result[3], 'descriptions':result[4], 'author_id':result[5]}
                recipes.append(dict_)
            cursor.close()
            print(recipes)
    
    user = session.get('user_email', '')
    return render_template("home.html", user_name = user, recipes = recipes)

@app.route("/<name>")
def user(name):
    return f"Hello {name}!"

@app.route("/login")
def login():
    user = session.get('user_email', '')
    return render_template("login.html", user_name = user)

@app.route("/recommend")
def recommed():
    recipes = []
    id = session.get('id', '')
    if id != '':
        select_query = ''' select Recipes.recipe_id,Recipes.recipe_name,Recipes.cooking_time,Recipes.cooking_difficulty,Recipes.descriptions,Recipes.author_id
                            from
                            (select 
                            recipe_id
                            from
                            (select I_name, action 
                            from relationship 
                            where user_id = :user_id) R
                            join
                            (select 
                            made_of as recipe_id,
                            I_name
                            from made_of) M
                            on R.I_name = M.I_name
                            group by recipe_id
                            having count(case when action = 'dislike' then 1 else null end) = 0
                            order by count(case when action = 'like' then 1 else null end) desc
                            limit 3) recipe_like
                            join Recipes
                            on recipe_like.recipe_id = Recipes.recipe_id
                            left join rating
                            on recipe_like.recipe_id = rating.recipe_id
                            group by Recipes.recipe_id,Recipes.recipe_name,Recipes.cooking_time,Recipes.cooking_difficulty,Recipes.descriptions,Recipes.author_id
                            order by avg(case when rating_score is not null then rating_score else 0 end) desc
                        '''
        cursor = g.conn.execute(text(select_query), {'user_id': id}) 
        for result in cursor:
            dict_ = {'recipe_id':result[0], 'recipe_name':result[1], 'cooking_time':result[2], 'cooking_difficulty':result[3], 'descriptions':result[4], 'author_id':result[5]}
            recipes.append(dict_)
        cursor.close()
        if len(recipes) == 0:
            select_query = ''' select Recipes.*
                                from
                                (select recipe_id 
                                from rating
                                where recipe_id not in 
                                (select recipe_id
                                from
                                    (select I_name
                                    from relationship 
                                    where user_id = :user_id
                                    and action = 'dislike') R
                                    join
                                    (select 
                                    made_of as recipe_id,
                                    I_name
                                    from made_of) M
                                    on R.I_name = M.I_name
                                ) 
                                group by recipe_id 
                                order by avg(rating_score) desc 
                                limit 3) R
                                join Recipes
                                on R.recipe_id = Recipes.recipe_id
                            '''


            cursor2 = g.conn.execute(text(select_query), {'user_id': id}) 
            for result in cursor2:
                dict_ = {'recipe_id':result[0], 'recipe_name':result[1], 'cooking_time':result[2], 'cooking_difficulty':result[3], 'descriptions':result[4], 'author_id':result[5]}
                recipes.append(dict_)

    else:
        select_query = ''' select Recipes.*
                            from
                            (select recipe_id 
                            from rating 
                            group by recipe_id 
                            order by avg(rating_score) desc 
                            limit 3) R
                            join Recipes
                            on R.recipe_id = Recipes.recipe_id
                        '''
        cursor = g.conn.execute(text(select_query))
        for result in cursor:
            dict_ = {'recipe_id':result[0], 'recipe_name':result[1], 'cooking_time':result[2], 'cooking_difficulty':result[3], 'descriptions':result[4], 'author_id':result[5]}
            recipes.append(dict_)
        cursor.close()

    user = session.get('user_email', '')
    return render_template("recommend.html", user_name = user, recipes = recipes)

@app.route("/profile")
def profile():
    id = session.get('id', '')
    recipes = []
    if id != '':
        select_query = '''  select *
                            from
                            Recipes
                            where author_id = :author_id
                            '''
        cursor = g.conn.execute(text(select_query),{'author_id': id})
        for result in cursor:
            dict_ = {'recipe_id':result[0], 'recipe_name':result[1], 'cooking_time':result[2], 'cooking_difficulty':result[3], 'descriptions':result[4], 'author_id':result[5]}
            recipes.append(dict_)
        print('recipes',recipes)
        cursor.close()

    user = session.get('user_email', '')
    return  render_template("profile.html", user_name = user, recipes = recipes)

if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Mar/2024 15:48:06] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:06] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:09] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:09] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:11] "GET /recommend HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:11] "GET /favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:14] "GET /profile HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2024 15:48:14] "GET /favicon.ico HTTP/1.1" 200 -


recipes [{'recipe_id': 54, 'recipe_name': 'Old English Lemon-Cranberry Cookies', 'cooking_time': 11, 'cooking_difficulty': 2, 'descriptions': "Preheat an oven to 350 degrees F (175 degrees C). Grease 2 baking sheets.\nMix the flour, sugar, baking soda, walnuts, and cloves together in a large mixing bowl. Beat the egg, applesauce, lemon juice, vanilla, cranberries, lemon zest, and butter together in a separate bowl with an electric hand mixer; pour into the flour mixture and mix well. Drop spoonfuls of the dough 2 inches apart onto the prepared baking sheets.\nBake in the preheated oven until the edges are golden, about 10 minutes. Allow the cookies to cool on the baking sheet for 1 minute before removing to a wire rack to cool completely. Dust with the confectioners' sugar to serve.\n", 'author_id': 31}]
